In [1]:
import datetime
import pytz
import xbos_services_getter
import math

In [2]:
# HOST_ADDRESS FOR ALL MICROSERVICES
XBOS_MICROSERVICES_HOST_ADDRESS=None #"FILL_ME_IN"

# CREATE A STUB TO EACH MICROSERVICE THAT YOU USE 
# DO NOT CALL MORE THAN ONCE PER SESSION
# THIS STUB ONLY GETS CALLED ONCE TO ESTABLISH A CONNECTION TO THE SERVICE
discomfort_stub = xbos_services_getter.get_discomfort_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
hvac_consumption_stub = xbos_services_getter.get_hvac_consumption_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
temperature_bands_stub = xbos_services_getter.get_temperature_band_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
price_stub = xbos_services_getter.get_price_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
occupancy_stub = xbos_services_getter.get_occupancy_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
outdoor_temperature_historical_stub = xbos_services_getter.get_outdoor_temperature_historic_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
# outdoor_temperature_prediction_stub = xbos_services_getter.get_outdoor_temperature_prediction_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
building_zone_names_stub = xbos_services_getter.get_building_zone_names_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
indoor_data_historical_stub = xbos_services_getter.get_indoor_historic_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
indoor_temperature_prediction_stub = xbos_services_getter.get_indoor_temperature_prediction_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
meter_data_historical_stub = xbos_services_getter.get_meter_data_historical_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
optimizer_stub = xbos_services_getter.get_optimizer_stub(XBOS_MICROSERVICES_HOST_ADDRESS)

In [3]:
# ALTERNATIVELY YOU CAN DEFINE THE FOLLOWING ENVIRONMENT VARIABLES FOR THE SERVICES YOU NEED
# FOR EXAMPLE FOR BASH YOU CAN USE 

# export DISCOMFORT_HOST_ADDRESS="FILL_ME_IN"
# export HVAC_CONSUMPTION_HOST_ADDRESS="FILL_ME_IN"
# export TEMPERATURE_BANDS_HOST_ADDRESS="FILL_ME_IN"
# export PRICE_HOST_ADDRESS="FILL_ME_IN"
# export OCCUPANCY_HOST_ADDRESS="FILL_ME_IN"
# export OUTDOOR_TEMPERATURE_HISTORICAL_HOST_ADDRESS="FILL_ME_IN"
# export OUTDOOR_TEMPERATURE_PREDICTION_HOST_ADDRESS="FILL_ME_IN"
# export BUILDING_ZONE_NAMES_HOST_ADDRESS="FILL_ME_IN"
# export INDOOR_DATA_HISTORICAL_HOST_ADDRESS="FILL_ME_IN"
# export INDOOR_TEMPERATURE_PREDICTION_HOST_ADDRESS="FILL_ME_IN"
# export METER_DATA_HISTORICAL_HOST_ADDRESS="FILL_ME_IN"
# export OPTIMIZER_HOST_ADDRESS="FILL_ME_IN"

# THEN YOU CAN CREATE A STUB TO EACH MICROSERVICE THAT YOU USE WITHOUT PASSING IN AN ADDRESS
# THIS STUB ONLY GETS CALLED ONCE TO ESTABLISH A CONNECTION TO THE SERVICE 

# discomfort_stub = xbos_services_getter.get_discomfort_stub()
# hvac_consumption_stub = xbos_services_getter.get_hvac_consumption_stub()
# temperature_bands_stub = xbos_services_getter.get_temperature_band_stub()
# price_stub = xbos_services_getter.get_price_stub()
# occupancy_stub = xbos_services_getter.get_occupancy_stub()
# outdoor_temperature_historical_stub = xbos_services_getter.get_outdoor_historic_stub()
# outdoor_temperature_prediction_stub = xbos_services_getter.get_outdoor_prediction_stub()
# building_zone_names_stub = xbos_services_getter.get_building_zone_names_stub()
# indoor_data_historical_stub = xbos_services_getter.get_indoor_historic_stub()
# indoor_temperature_prediction_stub = xbos_services_getter.get_indoor_temperature_prediction_stub()
# meter_data_historical_stub = xbos_services_getter.get_meter_data_historical_stub()
# optimizer_stub = xbos_services_getter.get_optimizer_stub()

In [4]:
# GET A LIST OF ALL BUILDINGS 
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)
print("List of buildings:",buildings)

# GET ALL ZONES FOR EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    print("List of zones in building:",bldg,"are:",zones)
    
# ALTERNATIVELY YOU CAN GET A DICTIONARY OF ALL BUILDINGS AND ZONES WITH ONE CALL
buildings_zones = xbos_services_getter.get_all_buildings_zones(building_zone_names_stub)
print("All buildings and zones:",buildings_zones)

List of buildings: ['avenal-veterans-hall', 'avenal-recreation-center', 'orinda-community-center', 'local-butcher-shop', 'avenal-public-works-yard', 'hayward-station-1', 'jesse-turner-center', 'ciee', 'csu-dominguez-hills', 'berkeley-corporate-yard', 'south-berkeley-senior-center', 'avenal-movie-theatre', 'avenal-animal-shelter', 'north-berkeley-senior-center', 'word-of-faith-cc', 'hayward-station-8']
List of zones in building: avenal-veterans-hall are: ['hvac_zone_ac_3', 'hvac_zone_ac_1', 'hvac_zone_ac_4', 'hvac_zone_ac_6', 'hvac_zone_ac_5', 'hvac_zone_ac_2']
List of zones in building: avenal-recreation-center are: ['hvac_zone_large_room', 'hvac_zone_tech_center']
List of zones in building: orinda-community-center are: ['hvac_zone_ac_7', 'hvac_zone_rm7', 'hvac_zone_kinder_gym', 'hvac_zone_ac_6', 'hvac_zone_ac_3', 'hvac_zone_rm1', 'hvac_zone_ac_4', 'hvac_zone_ac_5', 'hvac_zone_rm6', 'hvac_zone_ac_1', 'hvac_zone_front_office', 'hvac_zone_ac_2', 'hvac_zone_rm2', 'hvac_zone_ac_8']
List of

In [ ]:
# GET HISTORICAL OR FORECAST PRICE FOR EACH BUILDING (UP TO 24 HOURS IN THE FUTURE)
# EXAMPLE TO RETRIEVE PRICE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

# ENERGY OR DEMAND PRICES
energy_price_type = 'ENERGY' 
demand_price_type = 'DEMAND'

# GET ALL AVAILABLE TARIFFS AND UTILITIES
all_tarrifs_and_utilities = xbos_services_getter.get_all_tariffs(price_stub)

# GET THE PRICE FOR A GIVEN TARIFF AND UTILITY
for tarrif_utility in all_tarrifs_and_utilities:
    print(tarrif_utility)
    energy_price = xbos_services_getter.get_price_utility_tariff(price_stub,tarrif_utility["utility"],tarrif_utility["tariff"],energy_price_type,start,end,window)
    demand_price = xbos_services_getter.get_price_utility_tariff(price_stub,tarrif_utility["utility"],tarrif_utility["tariff"],demand_price_type,start,end,window)
    print(energy_price)
    print(demand_price)

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET THE TARIFF AND UTILITY FOR A GIVEN BUILDING THEN USE THIS INFORMATION TO GET THE PRICE FOR THE GIVEN BUILDING
for bldg in buildings:
    tarrif_utility = xbos_services_getter.get_tariff_and_utility(price_stub,bldg)
    print(bldg,tarrif_utility)
    energy_price = xbos_services_getter.get_price_utility_tariff(price_stub,tarrif_utility["utility"],tarrif_utility["tariff"],energy_price_type,start,end,window)
    demand_price = xbos_services_getter.get_price_utility_tariff(price_stub,tarrif_utility["utility"],tarrif_utility["tariff"],demand_price_type,start,end,window)
    print(energy_price)
    print(demand_price)

# ALTERNATIVELY YOU CAN ALSO GET THE PRICE FOR A GIVEN BUILDING DIRECTLY
for bldg in buildings:
    print(bldg)
    energy_price = xbos_services_getter.get_price(price_stub,bldg,energy_price_type,start,end,window)
    demand_price = xbos_services_getter.get_price(price_stub,bldg,demand_price_type,start,end,window)
    print(energy_price)
    print(demand_price)

In [ ]:
# GET HISTORICAL BUILDING METER & GREENBUTTON DATA
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JULY 1ST 2018
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2018, month=7, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# DEFINE POINT AND AGGREGATION TYPES
electric_point_type = 'Building_Electric_Meter'
greenbutton_point_type = 'Green_Button_Meter'
aggregate = 'RAW' # MEAN, MAX, MIN, COUNT, SUM and RAW


# GET HISTORICAL BUILDING METER & GREENBUTTON DATA FOR EACH BUILDING
for bldg in buildings:
    print(bldg)
    # NO METER OR GREENBUTTON DATA FOR csu-dominguez-hills
    if bldg == "csu-dominguez-hills":
        print("no meter or greenbutton data for csu-dominguez-hills")
        continue
    # NO GREENBUTTON DATA FOR jesse-turner-center
    if bldg == "jesse-turner-center":
        try:
            electric_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,electric_point_type,aggregate,window)
            print(electric_meter_data)
        except Exception as e:
            print(e)
        continue
    try:
        electric_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,electric_point_type,aggregate,window)
        greenbutton_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,greenbutton_point_type,aggregate,window)
        print(electric_meter_data)
        print(greenbutton_meter_data)
    except Exception as e:
        print(e)

In [ ]:
# GET HISTORICAL OUTDOOR TEMPERATURE 
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET OUTDOOR TEMPERATURE FOR EACH BUILDING
for bldg in buildings:
    print(bldg)
    historical_outdoor_temperature = xbos_services_getter.get_outdoor_temperature_historic(outdoor_temperature_historical_stub,bldg,start,end,window)
    print(historical_outdoor_temperature)

In [ ]:
# GET OUTDOOR TEMPERATURE PREDICTION
# EXAMPLE TO PREDICT TEMPERATURE FOR ONE DAY AT ONE HOUR INTERVAL STARTING AN HOUR FROM NOW
future_start = pytz.timezone('US/Pacific').localize(datetime.datetime.now())+ datetime.timedelta(hours=1)
future_end = future_start + datetime.timedelta(days=1)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET OUTDOOR TEMPERATURE PREDICTION FOR EACH BUILDING
for bldg in buildings:
    print(bldg)
    outdoor_temperature_prediction = xbos_services_getter.get_outdoor_temperature_prediction(outdoor_temperature_prediction_stub,bldg,future_start,future_end,window)
    print(outdoor_temperature_prediction)

In [ ]:
# GET HISTORICAL HVAC DATA (INDOOR TEMPERATURE, HVAC STATE, COOLING SETPOINT, HEATING SETPOING)
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"
aggregate = 'RAW' # MEAN, MAX, MIN, COUNT, SUM and RAW


# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET HISTORICAL HVAC DATA FOR EACH ZONE IN EACH BUILDING 
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        print(bldg,zone)
        try:
            actions = xbos_services_getter.get_indoor_actions_historic(indoor_data_historical_stub,bldg,zone,start,end,window,aggregate)
            indoor_temperatures = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window,aggregate)
            setpoints = xbos_services_getter.get_indoor_setpoints_historic(indoor_data_historical_stub,bldg,zone,start,end,window,aggregate)
            print(actions)
            print(indoor_temperatures)
            print(setpoints["t_low"])
            print(setpoints["t_high"])
        except Exception as e:
            print(e)



In [ ]:
# GET OCCUPANCY DATA
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET OCCUPANCY FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        occupancy = xbos_services_getter.get_occupancy(occupancy_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(occupancy)

In [ ]:
# GET COMFORT BAND AND DO NOT EXCEED BAND DATA
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET COMFORT BAND AND DO NOT EXCEED BAND FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        comfort_band = xbos_services_getter.get_comfortband(temperature_bands_stub,bldg,zone,start,end,window)
        do_not_exceed_band = xbos_services_getter.get_do_not_exceed(temperature_bands_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(comfort_band)
        print(do_not_exceed_band)

In [ ]:
# GET A DISCOMFORT METRIC FOR A GIVEN TEMPERATURE, COMFORT BAND, AND OCCUPANCY
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"
aggregate = 'MEAN' # MEAN, MAX, MIN, COUNT, SUM and RAW


# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET A DISCOMFORT METRIC FOR EVERY ZONE IN EVERY BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        print(bldg,zone)
        # GET THE HISTORICAL INDOOR TEMPERATURE, COMFORT BAND, AND OCCUPANCY
        try:
            indoor_temperatures = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window,aggregate)
            comfort_band = xbos_services_getter.get_comfortband(temperature_bands_stub,bldg,zone,start,end,window)
            occupancies = xbos_services_getter.get_occupancy(occupancy_stub,bldg,zone,start,end,window)
            for temperature,temperature_low,temperature_high,occupancy in zip(indoor_temperatures["temperature"],comfort_band["t_low"],comfort_band["t_high"],occupancies["occupancy"]):
                if any(math.isnan(v) for v in [temperature,temperature_low,temperature_high,occupancy]):
                    continue
                # GET A DISCOMFORT METRIC FOR A GIVEN TEMPERATURE, COMFORT BAND, AND OCCUPANCY
                discomfort = xbos_services_getter.get_discomfort(discomfort_stub,'bldg',temperature,temperature_low,temperature_high,occupancy)
                print(temperature,temperature_low,temperature_high,occupancy,discomfort)
        except Exception as e:
            print(e)

In [ ]:
# GET HVAC CONSUMPTION (FACE PLATE VALUE)
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET HVAC CONSUMPTION FOR A GIVEN ZONE IN A GIVEN BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        hvac_consumption = xbos_services_getter.get_hvac_consumption(hvac_consumption_stub,bldg,zone)
        print(bldg,zone,hvac_consumption)

In [5]:
# GET INDOOR TEMPERATURE PREDICTION GIVEN AN HVAC ACTION (NO_ACTION, HEATING_ACTION, COOLING_ACTION, FAN, TWO_STAGE_HEATING_ACTION, TWO_STAGE_COOLING_ACTION)
# EXAMPLE TO PREDICT TEMPERATURE FOR ONE HOUR STARTING JAN 1ST 2019 AT 01:00:00
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(hours=3)
window = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# GET INDOOR TEMPERATURE PREDICTION FOR A ZONE IN A BUILDING GIVEN AN ACTION 
action = xbos_services_getter.NO_ACTION # NO_ACTION, HEATING_ACTION, COOLING_ACTION, FAN, TWO_STAGE_HEATING_ACTION, TWO_STAGE_COOLING_ACTION
for bldg in buildings:
    outdoor_temperatures = xbos_services_getter.get_outdoor_temperature_historic(outdoor_temperature_historical_stub,bldg,start,end,window)
    outdoor_temperature = outdoor_temperatures["temperature"][1]  
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    current_zones_temp = {}
    prev_zones_temp = {}
    for zone in zones:
        indoor_temp = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        prev_zones_temp[zone] = indoor_temp["temperature"][0]
        current_zones_temp[zone]= indoor_temp["temperature"][1]
    for zone in zones:
        # THESE BUILDINGS DON'T HAVE A THERMAL MODEL YET
        if(bldg=='csu-dominguez-hills' or bldg=='local-butcher-shop' or bldg=='avenal-recreation-center'):
            continue
        print(bldg,zone)
        temperature_prediction,time,unit = xbos_services_getter.get_indoor_temperature_prediction(indoor_temperature_prediction_stub,bldg,zone,start,action,current_zones_temp[zone],outdoor_temperature,prev_zones_temp[zone],current_zones_temp)
        print(temperature_prediction,time,unit)

avenal-veterans-hall hvac_zone_ac_3
70.06751932268598 2019-01-01 00:05:00-08:00 F
avenal-veterans-hall hvac_zone_ac_1
70.24129522629931 2019-01-01 00:05:00-08:00 F
avenal-veterans-hall hvac_zone_ac_4
70.4941664554306 2019-01-01 00:05:00-08:00 F
avenal-veterans-hall hvac_zone_ac_6
45.85542027469528 2019-01-01 00:05:00-08:00 F
avenal-veterans-hall hvac_zone_ac_5
44.52211162045123 2019-01-01 00:05:00-08:00 F
avenal-veterans-hall hvac_zone_ac_2
70.24031748819121 2019-01-01 00:05:00-08:00 F
orinda-community-center hvac_zone_ac_7
62.39148904593819 2019-01-01 00:05:00-08:00 F
orinda-community-center hvac_zone_rm7
62.23823151501719 2019-01-01 00:05:00-08:00 F
orinda-community-center hvac_zone_kinder_gym
62.076895942894026 2019-01-01 00:05:00-08:00 F
orinda-community-center hvac_zone_ac_6
62.22290234905183 2019-01-01 00:05:00-08:00 F
orinda-community-center hvac_zone_ac_3
62.930822697616634 2019-01-01 00:05:00-08:00 F
orinda-community-center hvac_zone_rm1
62.08888036941752 2019-01-01 00:05:00-0

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "did not fetch data from pymortar with query: SELECT ?temp WHERE {
        ?temp rdf:type brick:Weather_Temperature_Sensor .
    };"
	debug_error_string = "{"created":"@1559028431.619260000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"did not fetch data from pymortar with query: SELECT ?temp WHERE {\n        ?temp rdf:type brick:Weather_Temperature_Sensor .\n    };","grpc_status":3}"
>

In [ ]:
# GET A SEQUENCE OF MPC OPTIMIZATION ACTIONS
# EXAMPLE TO GET ACTIONS (BASED ON MPC OPTIMIZATION) FOR FOUR HOURS STARTING APRIL 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=4, day=1, hour=0, minute=0))
temp_end = start + datetime.timedelta(hours=1)
opt_end = start + datetime.timedelta(hours=4)
window = "15m"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# RATIO BETWEEN DISCOMFORT AND COST
lambda_val = 0.5 # float between 0 and 1. The lambda value to balance cost and discomfort.

# GET AN MPC OPTIMIZATION ACTION FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    print(bldg)
    starting_temperatures = {}
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        indoor_temperatures = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,temp_end,window)
        starting_temperatures[zone]=indoor_temperatures["temperature"][0]
    # OPTIMIZE ONE ZONE AT A TIME
    for zone in zones:
        try:
            zone_actions = xbos_services_getter.get_mpc_optimization(optimizer_stub,bldg,[zone],start,opt_end,window,lambda_val,starting_temperatures)
            print(zone_actions)
        except Exception as e:
            print(zone)
            print(e)
    # ALTERNATIVELY YOU CAN OPTIMIZE ALL ZONES TOGETHER BUT THIS TAKES A LONG TIME TO RUN
#     try:
#         all_zones_actions = xbos_services_getter.get_mpc_optimization(optimizer_stub,bldg,zones,start,opt_end,window,lambda_val,starting_temperatures)    
#         print(all_zones_actions)
#     except Exception as e:
#         print(bldg,zones)
#         print(e)

In [4]:
# GET SIMULATION RESULTS
# EXAMPLE TO GET ACTIONS (BASED ON MPC OPTIMIZATION) FOR FOUR HOURS STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
temp_end = start + datetime.timedelta(hours=1)
opt_end = start + datetime.timedelta(hours=4)
window = "15m"
forecasting_horizon = "1h"

# GET ALL BUILDINGS
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)

# RATIO BETWEEN DISCOMFORT AND COST
lambda_val = 0.5 # float between 0 and 1. The lambda value to balance cost and discomfort.

# GET AN MPC OPTIMIZATION ACTION FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    starting_temperatures = {}
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        indoor_temperatures = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,temp_end,window)
        starting_temperatures[zone]=indoor_temperatures["temperature"][0]
    # OPTIMIZE ONE ZONE AT A TIME
    for zone in zones:
        print(bldg,zone)
        zone_actions, zone_temperatures = xbos_services_getter.get_mpc_simulation(optimizer_stub,bldg,[zone],start,opt_end,window, forecasting_horizon, lambda_val,starting_temperatures)
        print(zone_actions)
#     # ALTERNATIVELY YOU CAN OPTIMIZE ALL ZONES TOGETHER BUT THIS TAKES A LONG TIME TO RUN
#     print(bldg,zones)
#     all_zones_actions = xbos_services_getter.get_mpc_optimization(optimizer_stub,bldg,zones,start,opt_end,window,lambda_val,starting_temperatures)

avenal-veterans-hall hvac_zone_ac_3
[{'hvac_zone_ac_3': actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
}]
avenal-veterans-hall hvac_zone_ac_1
[{'hvac_zone_ac_1': actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
actions: 0
}]
avenal-veterans-hall hvac_zone_ac_4


KeyboardInterrupt: 

Exception ignored in: 'grpc._cython.cygrpc._next'
KeyboardInterrupt


KeyError: None

In [5]:
zone_temperatures

[{'hvac_zone_ac_1': temperatures: 75.0
  temperatures: 75.02061753391817
  temperatures: 75.01468062703296
  temperatures: 75.00546622662098
  temperatures: 74.9896566243569
  temperatures: 75.00409947356894
  temperatures: 75.0155930741934
  temperatures: 75.02955116949924
  temperatures: 75.03463848422876
  temperatures: 75.02797611295951
  temperatures: 74.99537143003059
  temperatures: 74.99646805956446
  temperatures: 74.99044565532846
  temperatures: 74.97325715005108
  temperatures: 74.96941639723593
  temperatures: 74.95921427827781
  temperatures: 74.94287523444403}]

In [1]:
import datetime
import pytz
import xbos_services_getter

start = pytz.timezone("US/Pacific").localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=120)
window = "15m"
building = "avenal-veterans-hall"
forecasting_horizon = '1d'
lambda_val = 0.5

print(start)

zn_stub = xbos_services_getter.xbos_services_getter.get_building_zone_names_stub('ms.xbos.io:9001')
zones = xbos_services_getter.xbos_services_getter.get_zones(zn_stub, building)
print(zones)

starting_temperatures = {k: 70 for k in zones}

optimizer_stub = xbos_services_getter.get_optimizer_stub('ms.xbos.io:9001')
res = xbos_services_getter.xbos_services_getter.get_mpc_simulation(optimizer_stub, building, zones, start, end, window, forecasting_horizon, lambda_val, starting_temperatures, unit='F', num_runs=1)

2019-01-01 00:00:00-08:00
['hvac_zone_ac_3', 'hvac_zone_ac_1', 'hvac_zone_ac_4', 'hvac_zone_ac_6', 'hvac_zone_ac_5', 'hvac_zone_ac_2']


_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: <_Rendezvous of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Not enough data for given action to get error."
	debug_error_string = "{"created":"@1559027720.741157264","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Not enough data for given action to get error.","grpc_status":3}"
>"
	debug_error_string = "{"created":"@1559027720.716683000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Exception calling application: <_Rendezvous of RPC that terminated with:\n\tstatus = StatusCode.INVALID_ARGUMENT\n\tdetails = "Not enough data for given action to get error."\n\tdebug_error_string = "{"created":"@1559027720.741157264","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line